In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn import metrics
import numpy as npw
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.preprocessing import MinMaxScaler





# Read the CSV file
data = pd.read_csv("../data/data/aapl_raw_data.csv")
data.shape
data.tail(1)

data = data.iloc[:10747]

data.tail(1)


data.isnull().sum()
data=data.fillna(0)  # Filling null values with zero
data.isnull().sum()



#data["date"] = data["date"].astype(float)
data["open"] = data["open"].astype(float)
data["high"] = data["high"].astype(float)
data["low"] = data["low"].astype(float)
data["volume"] = data["volume"].astype(float)
data["adjusted_close"] = data["adjusted_close"].astype(float)
data["change_percent"] = data["change_percent"].astype(float)
data["avg_vol_20d"] = data["avg_vol_20d"].astype(float)

data["close"] = data["close"].astype(float)

#print(data.tail(1))

In [2]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Convert data to numpy arrays
x_data = data[['open', 'high', 'low', 'volume', 'adjusted_close', 'change_percent', 'avg_vol_20d']].values
y_data = data["close"].values.reshape(-1, 1)  # Reshape y_data if it's 1D

# Initialize the scalers
scaler_x = StandardScaler()
scaler_y = StandardScaler()

# Fit the scaler to the training data and transform both training and test data
x_data_scaled = scaler_x.fit_transform(x_data)
y_data_scaled = scaler_y.fit_transform(y_data)

# Convert scaled data to Tensors
x_feature_tensors = torch.tensor(x_data_scaled, dtype=torch.float32)
y_feature_tensors = torch.tensor(y_data_scaled, dtype=torch.float32)

# Split the training data into training and temporary sets
x_train, x_temp, y_train, y_temp = train_test_split(x_feature_tensors, y_feature_tensors, test_size=0.2, random_state=42)

# Split the temporary set into validation and test sets
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)

# Convert scaled labels back to numpy arrays
y_train = y_train.numpy()
y_val = y_val.numpy()
y_test = y_test.numpy()



# Now you can use x_train_scaled, x_val, x_test, y_train_scaled, y_val_scaled, y_test_scaled for your model training


In [3]:
"""
# Last 5 values of x
print("Last 5 values of x:")
print(x_data[-5:])
print("\n")

# Last 5 values of y
print("Last 5 values of y:")
print(y_data[-5:])
"""

'\n# Last 5 values of x\nprint("Last 5 values of x:")\nprint(x_data[-5:])\nprint("\n")\n\n# Last 5 values of y\nprint("Last 5 values of y:")\nprint(y_data[-5:])\n'

In [4]:
"""
# Inverse transform the scaled data to get original scales
x_original = scaler_x.inverse_transform(x_feature_tensors.numpy())
y_original = scaler_y.inverse_transform(y_feature_tensors.numpy())

# Print the last 5 values of original x and y
print("Last 5 values of original x:")
print(x_original[-5:])
print("\n")

print("Last 5 values of original y:")
print(y_original[-5:])
"""

'\n# Inverse transform the scaled data to get original scales\nx_original = scaler_x.inverse_transform(x_feature_tensors.numpy())\ny_original = scaler_y.inverse_transform(y_feature_tensors.numpy())\n\n# Print the last 5 values of original x and y\nprint("Last 5 values of original x:")\nprint(x_original[-5:])\nprint("\n")\n\nprint("Last 5 values of original y:")\nprint(y_original[-5:])\n'

In [5]:

learning_rate = 0.001
window_size = 50
test_window_size = 50
hidden_dim = 64
n_layers = 4
batch_evaluation_frequency = 10
epochs = len(x_train) - window_size
batch_size = 1
input_size = x_train.shape[1]  # Input size based on your dataset
output_size = 1  # Output size (for regression task)

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_dim, n_layers, output_size):
        super(LSTMModel, self).__init__()

        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        self.lstm = nn.LSTM(input_size, hidden_dim, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_size)

    def forward(self, x, hidden):
        batch_size = x.size(0)
        out, hidden = self.lstm(x, hidden)
        out = self.fc(out[:, -1, :])
        return out, hidden

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        return hidden


In [6]:
import torch
import torch.optim as optim
import torch.nn as nn
from sklearn.model_selection import train_test_split
import random
import pandas as pd

# Assuming x_train_scaled, y_train_scaled, x_val_scaled, y_val_scaled, x_test_scaled, y_test_scaled,
# input_size, output_size, test_window_size, scaler are available


# Hyperparameter search space
search_space = {
    'learning_rate': [0.00005],
    'window_size': [ 5],
    'hidden_dim': [256],
    'n_layers': [11],
    'batch_evaluation_frequency': [4]
}

#Best Parameters: {'learning_rate': 5e-05, 'window_size': 5, 'hidden_dim': 256, 'n_layers': 11, 'batch_evaluation_frequency': 4, 'test_loss': 0.41702309250831604}
#              'n_layers': 11, 'batch_evaluation_frequency': 4, 'test_loss': 6463.20556640625}


# Number of random search iterations
num_iterations = 10

best_params = None

for _ in range(num_iterations):
    params = {
        'learning_rate': random.choice(search_space['learning_rate']),
        'window_size': random.choice(search_space['window_size']),
        'hidden_dim': random.choice(search_space['hidden_dim']),
        'n_layers': random.choice(search_space['n_layers']),
        'batch_evaluation_frequency': random.choice(search_space['batch_evaluation_frequency'])
    }

    model = LSTMModel(input_size, params['hidden_dim'], params['n_layers'], output_size)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=params['learning_rate'])

    # Training using Walk-Forward Validation
    for i in range(params['window_size'], len(x_train)):
        optimizer.zero_grad()

        start_idx = i - params['window_size']
        end_idx = i
        x_window = torch.tensor(x_train[start_idx:end_idx], dtype=torch.float32)
        y_window = torch.tensor(y_train[start_idx:end_idx], dtype=torch.float32)
        x_window = x_window.view(1, params['window_size'], input_size)

        hidden = model.init_hidden(1)
        outputs, hidden = model(x_window, hidden)
        loss = criterion(outputs, y_window)

        loss.backward()
        optimizer.step()

        if i % params['batch_evaluation_frequency'] == 0:
            with torch.no_grad():
                x_val_window = torch.tensor(x_val[:params['window_size']], dtype=torch.float32)
                y_val_window = torch.tensor(y_val[:params['window_size']], dtype=torch.float32)
                x_val_window = x_val_window.view(1, params['window_size'], input_size)

                hidden = model.init_hidden(1)
                val_outputs, _ = model(x_val_window, hidden)
                val_loss = criterion(val_outputs, y_val_window)

                if i % 1000 == 0:  # Print every 1000 iterations
                    print(f"Iteration {i}, Train Loss: {loss.item()}, Validation Loss: {val_loss.item()}")

    # Test set evaluation
    with torch.no_grad():
        x_test_window = torch.tensor(x_test[:params['window_size']], dtype=torch.float32)
        y_test_window = torch.tensor(y_test[:params['window_size']], dtype=torch.float32)
        x_test_window = x_test_window.view(1, params['window_size'], input_size)

        hidden = model.init_hidden(1)
        test_outputs, _ = model(x_test_window, hidden)
        test_loss = criterion(test_outputs, y_test_window)

    print(f"Test Loss: {test_loss.item()}")

    # Update the best_params if the current model performs better on the test set
    if best_params is None:
        best_params = params
        best_params['test_loss'] = test_loss.item()
    elif test_loss < best_params['test_loss']:
        best_params = params
        best_params['test_loss'] = test_loss.item()

print(f"Best Parameters: {best_params}")

C:\Users\pitr7\AppData\Local\Temp\ipykernel_11712\3789981942.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_window = torch.tensor(x_train[start_idx:end_idx], dtype=torch.float32)
c:\Projectarbeit-Dow-Jones-Index\.venv\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([5, 1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\pitr7\AppData\Local\Temp\ipykernel_11712\3789981942.py:62: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_val_window = torch.tensor(x_va

Iteration 1000, Train Loss: 1.0496785640716553, Validation Loss: 2.250458240509033
Iteration 2000, Train Loss: 2.076263427734375, Validation Loss: 2.236117124557495
Iteration 3000, Train Loss: 0.23874036967754364, Validation Loss: 2.2375779151916504
Iteration 4000, Train Loss: 0.3783578872680664, Validation Loss: 2.2188756465911865
Iteration 5000, Train Loss: 0.30434587597846985, Validation Loss: 2.2213265895843506
Iteration 6000, Train Loss: 0.16998176276683807, Validation Loss: 2.2246384620666504
Iteration 7000, Train Loss: 0.2303958386182785, Validation Loss: 2.2296674251556396
Iteration 8000, Train Loss: 0.33391833305358887, Validation Loss: 2.2227959632873535


C:\Users\pitr7\AppData\Local\Temp\ipykernel_11712\3789981942.py:75: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_test_window = torch.tensor(x_test[:params['window_size']], dtype=torch.float32)


Test Loss: 0.4176923334598541
Iteration 1000, Train Loss: 1.0503355264663696, Validation Loss: 2.251126289367676
Iteration 2000, Train Loss: 2.078392744064331, Validation Loss: 2.236886501312256
Iteration 3000, Train Loss: 0.23733869194984436, Validation Loss: 2.2383456230163574
Iteration 4000, Train Loss: 0.377674400806427, Validation Loss: 2.21925687789917
Iteration 5000, Train Loss: 0.30314236879348755, Validation Loss: 2.2219700813293457
Iteration 6000, Train Loss: 0.1688717156648636, Validation Loss: 2.225498676300049
Iteration 7000, Train Loss: 0.22905020415782928, Validation Loss: 2.230363368988037
Iteration 8000, Train Loss: 0.33254170417785645, Validation Loss: 2.2233097553253174
Test Loss: 0.41825518012046814
Iteration 1000, Train Loss: 1.0484178066253662, Validation Loss: 2.2493083477020264
Iteration 2000, Train Loss: 2.0740513801574707, Validation Loss: 2.235318899154663
Iteration 3000, Train Loss: 0.2400786429643631, Validation Loss: 2.2368478775024414
Iteration 4000, Trai

In [10]:
# ... (previous code)

# Test set evaluation
with torch.no_grad():
    x_test_window = torch.tensor(x_test[-params['window_size']:], dtype=torch.float32)
    y_test_window = torch.tensor(y_test[-params['window_size']:], dtype=torch.float32)
    x_test_window = x_test_window.view(1, params['window_size'], input_size)

    hidden = model.init_hidden(1)
    _, _ = model(x_test_window, hidden)  # We only need to run the model to get the hidden states

# Convert the scaled features back to original scale
original_last_window_features = scaler_x.inverse_transform(x_test[-params['window_size']:])

# Print the last window of features that the model uses in original scale
last_window_features = pd.DataFrame(original_last_window_features, columns=['open', 'high', 'low', 'volume', 'adjusted_close', 'change_percent', 'avg_vol_20d'])


#print("\nLast Window of Features Used by the Model (Original Scale):")
#print(last_window_features)
# ... (rest of the code)



Last Window of Features Used by the Model (Original Scale):
         open        high         low        volume  adjusted_close  \
0  606.180410  616.279988  606.060000  2.168320e+07         18.6277   
1   31.875203   31.875196   31.628800  1.057993e+05          0.1094   
2   39.244802   39.995203   38.998398  7.496000e+05          0.2775   
3   34.742399   35.369603   34.630403  9.310999e+05          0.1224   
4  568.100409  569.500412  563.379596  7.286000e+06         17.6792   

   change_percent   avg_vol_20d  
0            1.24  7.270806e+08  
1           -0.82  7.940575e+07  
2            0.95  1.257654e+08  
3            2.17  1.837226e+08  
4           -0.66  3.518187e+08  


C:\Users\pitr7\AppData\Local\Temp\ipykernel_11712\2384865010.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_test_window = torch.tensor(x_test[-params['window_size']:], dtype=torch.float32)


In [12]:
# ... (previous code)

# Test set evaluation
with torch.no_grad():
    x_test_window = torch.tensor(x_test[:params['window_size']], dtype=torch.float32)
    y_test_window = torch.tensor(y_test[:params['window_size']], dtype=torch.float32)
    x_test_window = x_test_window.view(1, params['window_size'], input_size)

    hidden = model.init_hidden(1)
    test_outputs, _ = model(x_test_window, hidden)
    test_loss = criterion(test_outputs, y_test_window)

# Print the last test window and compare with the last 5 entries of the original data
print("Last Test Window:")
print("Predicted (scaled):", test_outputs.numpy())
print("Actual (scaled):", y_test_window.numpy())

# Convert the scaled predictions back to original scale
predicted_values = scaler_y.inverse_transform(test_outputs.numpy())
actual_values = scaler_y.inverse_transform(y_test_window.numpy())

print("\nLast 5 Entries of Original Data:")
print(data.tail(5)[['open', 'high', 'low', 'volume', 'adjusted_close', 'change_percent', 'avg_vol_20d']])
print("Actual (original):", actual_values[-5:])
print("Predicted (original):", predicted_values[-5:])

# ... (rest of the code)


Last Test Window:
Predicted (scaled): [[0.03434191]]
Actual (scaled): [[-0.6992109 ]
 [ 0.6115489 ]
 [ 0.5079761 ]
 [-0.66462284]
 [-0.681237  ]]

Last 5 Entries of Original Data:
         open    high     low      volume  adjusted_close  change_percent  \
10742  193.33  194.44  192.92  37283200.0        193.3589            0.45   
10743  193.67  195.64  193.32  47471900.0        194.2377            0.45   
10744  196.02  197.20  192.55  47460200.0        192.9594           -0.66   
10745  194.67  196.63  194.14  48291400.0        195.5659            1.35   
10746  196.06  196.49  195.26  38824100.0        196.1851            0.32   

       avg_vol_20d  
10742   52369165.0  
10743   52206220.0  
10744   52018390.0  
10745   52115595.0  
10746   49803320.0  
Actual (original): [[ 20.003197]
 [192.75    ]
 [179.09999 ]
 [ 24.561607]
 [ 22.372002]]
Predicted (original): [[116.67911]]


C:\Users\pitr7\AppData\Local\Temp\ipykernel_11712\4261605350.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_test_window = torch.tensor(x_test[:params['window_size']], dtype=torch.float32)
c:\Projectarbeit-Dow-Jones-Index\.venv\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([5, 1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
